In [1]:
import boto3

In [2]:
import os
os.environ.setdefault('AWS_PROFILE', 'ghactivity')

'ghactivity'

In [3]:
dynamodb = boto3.resource('dynamodb')
job_details_table = dynamodb.Table('jobs')

In [4]:
job_details_table.delete_item(Key={'job_id': 'ghactivity_ingest'}) 

{'ResponseMetadata': {'RequestId': '419V7QU6IPPPUPOE7RJOQ60A1BVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 14 Sep 2023 12:14:30 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '419V7QU6IPPPUPOE7RJOQ60A1BVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [5]:
job_details = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'Ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days': 3
}

In [6]:
job_details_table.put_item(Item=job_details)

{'ResponseMetadata': {'RequestId': 'LKNU9K6NL31DTONT33KQ710O07VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 14 Sep 2023 12:15:04 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'LKNU9K6NL31DTONT33KQ710O07VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [7]:
def get_job_details(job_name):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('jobs')
    job_details = table.get_item(Key={'job_id': job_name})['Item']
    return job_details

In [8]:
job_details = get_job_details('ghactivity_ingest')
job_details

{'job_description': 'Ingest ghactivity data to s3',
 'is_active': 'Y',
 'job_id': 'ghactivity_ingest',
 'baseline_days': Decimal('3')}

In [9]:
from datetime import datetime as dt

In [10]:
from datetime import timedelta as td

In [11]:
dt.now()

datetime.datetime(2023, 9, 14, 13, 16, 42, 979694)

In [14]:
dt.now().date()

datetime.date(2023, 9, 14)

In [15]:
dt.now().date() - td(days=3)

datetime.date(2023, 9, 11)

In [16]:
f'{dt.strftime(dt.now().date() - td(days=3), "%Y-%m-%d")}-0.json.gz'

'2023-09-11-0.json.gz'

In [17]:
next_file = f'{dt.strftime(dt.now().date() - td(days=3), "%Y-%m-%d")}-0.json.gz'

In [18]:
next_file

'2023-09-11-0.json.gz'

In [19]:
import requests

In [20]:
res = requests.get(f'https://data.gharchive.org/{next_file}')

In [21]:
file = open(f'data/{next_file}', 'wb')

In [22]:
file.write(res.content)

70303756

In [23]:
file.close()

In [53]:
!ls -ltr data/

total 147608
-rw-r--r--  1 itversity  staff       127 Jun  7 10:46 2022-06-07-8.json.gz
-rw-r--r--  1 itversity  staff  41575819 Jun  7 11:02 2022-06-05-8.json.gz
-rw-r--r--  1 itversity  staff  33989636 Jun  8 04:22 2022-06-05-0.json.gz


In [24]:
import pandas as pd

In [25]:
df = pd.read_json(
    f'data/{next_file}', 
    lines=True, 
    orient='records'
)

In [26]:
df

,id,type,actor,repo,payload,public,created_at,org
0,31726112216,PushEvent,"{'id': 111329684, 'login': 'B74LABgit', 'displ...","{'id': 684638644, 'name': 'B74LABgit/CAM', 'ur...","{'repository_id': 684638644, 'push_id': 149889...",True,2023-09-11 00:00:00+00:00,NaN
1,31726112221,PushEvent,"{'id': 125916451, 'login': 'Sameshnee21', 'dis...","{'id': 677840347, 'name': 'Sameshnee21/2490510...","{'repository_id': 677840347, 'push_id': 149889...",True,2023-09-11 00:00:00+00:00,NaN
2,31726112231,PushEvent,"{'id': 1794609, 'login': 'carlosleonardo', 'di...","{'id': 687203378, 'name': 'carlosleonardo/ToDo...","{'repository_id': 687203378, 'push_id': 149889...",True,2023-09-11 00:00:00+00:00,NaN
3,31726112235,PullRequestEvent,"{'id': 64147848, 'login': 'yungsamd17', 'displ...","{'id': 677023791, 'name': 'yungsamd17/paycheck...","{'action': 'closed', 'number': 4, 'pull_reques...",True,2023-09-11 00:00:00+00:00,NaN
4,31726112236,PullRequestEvent,"{'id': 41898282, 'login': 'github-actions[bot]...","{'id': 470320620, 'name': 'davidfletcher7/docu...","{'action': 'opened', 'number': 30, 'pull_reque...",True,2023-09-11 00:00:00+00:00,NaN
...,...,...,...,...,...,...,...,...
156176,31726665706,PushEvent,"{'id': 41898282, 'login': 'github-actions[bot]...","{'id': 684091149, 'name': 'Matthias-Zax/Matthi...","{'repository_id': 684091149, 'push_id': 149892...",True,2023-09-11 00:59:59+00:00,NaN
156177,31726665708,DeleteEvent,"{'id': 49699333, 'login': 'dependabot[bot]', '...","{'id': 336129858, 'name': 'nczitzk/RSSHub', 'u...",{'ref': 'dependabot/github_actions/actions/che...,True,2023-09-11 00:59:59+00:00,NaN
156178,31726665712,PushEvent,"{'id': 77761206, 'login': 'mentalcaries', 'dis...","{'id': 665527349, 'name': 'mentalcaries/connec...","{'repository_id': 665527349, 'push_id': 149892...",True,2023-09-11 00:59:59+00:00,NaN
156179,31726665715,PushEvent,"{'id': 41898282, 'login': 'github-actions[bot]...","{'id': 670569644, 'name': 'BiologyHazard/Biolo...","{'repository_id': 670569644, 'push_id': 149892...",True,2023-09-11 00:59:59+00:00,NaN


In [27]:
df.count()

id            156181
type          156181
actor         156181
repo          156181
payload       156181
public        156181
created_at    156181
org            29817
dtype: int64

In [28]:
df.columns

Index(['id', 'type', 'actor', 'repo', 'payload', 'public', 'created_at',
       'org'],
      dtype='object')

In [29]:
df.dtypes

id                          int64
type                       object
actor                      object
repo                       object
payload                    object
public                       bool
created_at    datetime64[ns, UTC]
org                        object
dtype: object

In [33]:
# Check for duplicate values in the specified column
duplicates = df[df.duplicated(subset=column_name, keep=False)]

# Count the number of duplicate values
duplicate_count = len(duplicates)

# Print the duplicate values and their count
print("Duplicate Values:")
print(duplicates)
print(f"Number of Duplicate Values in '{column_name}': {duplicate_count}")

Duplicate Values:
Empty DataFrame
Columns: [id, type, actor, repo, payload, public, created_at, org]
Index: []
Number of Duplicate Values in 'id': 0
